## Colab Prep

Execute the following code cells to whenever you open/restart the notebook in Google Colab.

In [ ]:
!pip install "polars[all]"

In [ ]:
!wget https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/sample_data.zip

In [ ]:
!unzip ./sample_data.zip

In [ ]:
!pip install more_polars

# Concatenating Tables with Set-Like Operations

One of the two way of combining two tables is to stack one table on top of the other.  When stacking two tables on top of one another, we need to decide

1. If we combine columns based on position or name (and if combining by name, what do we do with mismatches?)
2. How to decide which rows to keep.  In this case, we will take some guidance from SQL clauses.

## Three Types of Operations

* **Union:** Keeps rows from either table.
* **Intersection:** Only keeps common columns
* **Set Difference/Except:** Keep rows from the left table *except* those in the right table.

## Set Operations in Action 

<img src="./img/table_verbs_set.gif" width=800>

## All Operations Match by Position

All operations

* Match columns by position
* Require same number/type of columns

## Distinct Versus All

* **UNION/INTERSECT/SET DIFFERENCE** are **DISTINCT**
    * Only keeps distinct rows, removing duplicates.
* **UNION ALL/INTERSECT ALL/SET DIFFERENCE ALL**
    * Keeps duplicate rows

In [2]:
import polars as pl
pl.Config.with_columns_kwargs = True

In [3]:
sales_apr = pl.read_csv('./sample_data/auto_sales_apr.csv')
sales_apr

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9


In [4]:
sales_may = pl.read_csv('./sample_data/auto_sales_may.csv')
sales_may

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",20,14,6,24
"""Yolanda""",19,10,28,17
"""Xerxes""",11,27,17,9


In [5]:
sales_june = pl.read_csv('./sample_data/auto_sales_june.csv')
sales_june

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",18,19,17,12
"""Bob""",20,15,8,23
"""Yolanda""",21,9,22,19
"""Xerxes""",12,25,19,8


## The general `polars` SQL interface

We can use a `SQL` query to process `polars` tables by

1. Loading all tables, e.g. using `read_csv`,
2. Passing a SQL query to `pl.sql` [Use a multi-line string, e.g., `""" ... """`
3. Use `collect` to access the result.

### Example - Average weekly sales of Compact cars from April

Assuming there were 4 weeks in April, let's compute each salesperson's average weekly sales using a `SQL` query.

In [23]:
pl.sql("""
SELECT AVG(Compact) AS mean_compact
from sales_apr
""")

In [24]:
pl.sql("""
SELECT AVG(Compact) AS mean_compact
from sales_apr
""").collect()

mean_compact
f64
18.0


## Performing `UNIONS` in `polars`

Currently, there isn't a Python interface for `UNION`, but we can perform this action as a `SQL` query.

#### Example 1 - Union all sales from april-june

In [25]:
pl.sql("""
SELECT * from sales_apr
UNION ALL SELECT * from sales_may
UNION ALL SELECT * from sales_june
""").collect()

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9
"""Ann""",22,18,15,12
…,…,…,…,…
"""Xerxes""",11,27,17,9
"""Ann""",18,19,17,12
"""Bob""",20,15,8,23


#### Example 2 - Find all salespersons with at least 40 total compact sales across april-june.

In [26]:
pl.sql("""
SELECT u.Salesperson, SUM(u.Compact) AS Total_Compact from 
    (
    SELECT * from sales_apr
    UNION ALL SELECT * from sales_may
    UNION ALL SELECT * from sales_june
    ) AS u
GROUP BY u.Salesperson
HAVING Total_Compact >= 40
""").collect()

Salesperson,Total_Compact
str,i64
"""Yolanda""",59
"""Ann""",62
"""Bob""",59


### Saving the result of a `SQL` query

Note that a `pl.sql(...).collect()` returns a `polars` dataframe that can be used subsequently.

In [27]:
combined_sales = pl.sql("""
                        SELECT * from sales_apr
                        UNION ALL SELECT * from sales_may
                        UNION ALL SELECT * from sales_june
                        """).collect()
combined_sales

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9
"""Ann""",22,18,15,12
…,…,…,…,…
"""Xerxes""",11,27,17,9
"""Ann""",18,19,17,12
"""Bob""",20,15,8,23


In [20]:
type(combined_sales)

polars.dataframe.frame.DataFrame

## Other ways to combine tables in `polars`

* Use `vstack` to perform a union on 2 tables.
* Use `pl.concat` to perform a union of 3+ tables.

In [26]:
sales_apr.vstack(sales_may)

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9
"""Ann""",22,18,15,12
"""Bob""",20,14,6,24
"""Yolanda""",19,10,28,17
"""Xerxes""",11,27,17,9


## `df.vstack` is NOT distinct

You need to perform a `unique` after the union to get distinct rows.

In [27]:
sales_may.vstack(sales_apr).unique()

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",20,14,6,24
"""Yolanda""",19,10,28,17
"""Xerxes""",11,27,17,9
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9


In [28]:
sales_may.vstack(sales_apr).unique(keep='last')

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Bob""",20,14,6,24
"""Yolanda""",19,10,28,17
"""Xerxes""",11,27,17,9
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9


## Columns are stacked by column location/order!

In [29]:
(sales_apr.select(['Salesperson','Compact','Sedan','SUV','Truck'])
.vstack(sales_may.select(['Salesperson','SUV','Truck','Compact','Sedan']))
)

SchemaError: cannot vstack: because column names in the two DataFrames do not match for left.name='Compact' != right.name='SUV'

## Adding a month column

Another way to keep both of Ann's sales rows is adding a month column (which we should probably do anyway).

In [30]:
pl.Config.with_columns_kwargs = True

(sales_may
 .with_columns(month = 'May')
 .vstack(sales_apr
         .with_columns(month = 'April')
        )
)

Salesperson,Compact,Sedan,SUV,Truck,month
str,i64,i64,i64,i64,str
"""Ann""",22,18,15,12,"""May"""
"""Bob""",20,14,6,24,"""May"""
"""Yolanda""",19,10,28,17,"""May"""
"""Xerxes""",11,27,17,9,"""May"""
"""Ann""",22,18,15,12,"""April"""
"""Bob""",19,12,17,20,"""April"""
"""Yolanda""",19,8,32,15,"""April"""
"""Xerxes""",12,23,18,9,"""April"""


## No `INTERSECT` or `DIFFERENCE` in `polars`

As of Fall 2022, `polars` lacks the either of these set operations.

## Combining multiple files using vstack

The first method of performing a union on more than two tables function is to dot-chain `vstack`.

#### Combining the raw files

In [49]:
(sales_apr
 .vstack(sales_may)
 .vstack(sales_june)
)

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9
"""Ann""",22,18,15,12
"""Bob""",20,14,6,24
"""Yolanda""",19,10,28,17
"""Xerxes""",11,27,17,9
"""Ann""",18,19,17,12


#### Adding a month column to each

This approach becomes messy when processing each table, as the processing of all subsequent table most be nested inside `vstack`.

In [52]:
(sales_apr
 .with_columns(month = 'April')
 .vstack(sales_may
         .with_columns(month = 'May')
        )
 .vstack(sales_june
         .with_columns(month = 'June')
        )
)

Salesperson,Compact,Sedan,SUV,Truck,month
str,i64,i64,i64,i64,str
"""Ann""",22,18,15,12,"""April"""
"""Bob""",19,12,17,20,"""April"""
"""Yolanda""",19,8,32,15,"""April"""
"""Xerxes""",12,23,18,9,"""April"""
"""Ann""",22,18,15,12,"""May"""
"""Bob""",20,14,6,24,"""May"""
"""Yolanda""",19,10,28,17,"""May"""
"""Xerxes""",11,27,17,9,"""May"""
"""Ann""",18,19,17,12,"""June"""


## Combining multiple files using concatenate

Another way to perform unions on many files is the function `pl.concat` allows stacking any number of files with the same columns.

#### Combining the raw files

In [53]:
pl.concat([sales_apr, sales_may, sales_june])

Salesperson,Compact,Sedan,SUV,Truck
str,i64,i64,i64,i64
"""Ann""",22,18,15,12
"""Bob""",19,12,17,20
"""Yolanda""",19,8,32,15
"""Xerxes""",12,23,18,9
"""Ann""",22,18,15,12
"""Bob""",20,14,6,24
"""Yolanda""",19,10,28,17
"""Xerxes""",11,27,17,9
"""Ann""",18,19,17,12


#### Adding a month column to each

In [55]:
pl.concat([sales_apr.with_columns(month = 'April'),
           sales_may.with_columns(month = 'May'),
           sales_june.with_columns(month = 'June'),
           ])

Salesperson,Compact,Sedan,SUV,Truck,month
str,i64,i64,i64,i64,str
"""Ann""",22,18,15,12,"""April"""
"""Bob""",19,12,17,20,"""April"""
"""Yolanda""",19,8,32,15,"""April"""
"""Xerxes""",12,23,18,9,"""April"""
"""Ann""",22,18,15,12,"""May"""
"""Bob""",20,14,6,24,"""May"""
"""Yolanda""",19,10,28,17,"""May"""
"""Xerxes""",11,27,17,9,"""May"""
"""Ann""",18,19,17,12,"""June"""


## <font color="red"> Exercise 7.4.1</font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Read the April-August data frames,
2. Add the month column each data frame,
3. Use a `SQL` query to perform a `UNION`,
4. Use `df.vstack` to combine these data frames into one combined `df`, and
5. Use `pd.concat` to combine these data frames into one combined `df`.

In [38]:
# Your code here